In [1]:
import numpy as np
import pandas as pd
import gc
import time
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
%matplotlib inline

In [2]:
# baseline
import pandas as pd
import numpy as np

from glob import glob
from tqdm import tqdm
from scipy import interpolate

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, AveragePooling1D, GlobalAveragePooling1D

In [3]:
w_list = sorted(glob("Dataset/water_data/*.csv"))
w_list

['Dataset/water_data\\data_2012.csv',
 'Dataset/water_data\\data_2013.csv',
 'Dataset/water_data\\data_2014.csv',
 'Dataset/water_data\\data_2015.csv',
 'Dataset/water_data\\data_2016.csv',
 'Dataset/water_data\\data_2017.csv',
 'Dataset/water_data\\data_2018.csv',
 'Dataset/water_data\\data_2019.csv',
 'Dataset/water_data\\data_2020.csv',
 'Dataset/water_data\\data_2021.csv',
 'Dataset/water_data\\data_2022.csv']

In [4]:
pd.read_csv(w_list[0]).shape

(26496, 15)

In [5]:
pd.read_csv(w_list[0]).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26496 entries, 0 to 26495
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ymdhm       26496 non-null  object 
 1   swl         26495 non-null  float64
 2   inf         26495 non-null  float64
 3   sfw         26495 non-null  float64
 4   ecpc        26495 non-null  float64
 5   tototf      26495 non-null  float64
 6   tide_level  25720 non-null  float64
 7   wl_1018662  26496 non-null  float64
 8   fw_1018662  26496 non-null  float64
 9   wl_1018680  26496 non-null  float64
 10  fw_1018680  26496 non-null  float64
 11  wl_1018683  26496 non-null  float64
 12  fw_1018683  26496 non-null  float64
 13  wl_1019630  26496 non-null  float64
 14  fw_1019630  26496 non-null  float64
dtypes: float64(14), object(1)
memory usage: 3.0+ MB


In [6]:
with tf.device("/device:GPU:0"):
    train_data = []
    train_label = []
    num = 0

    for i in w_list[:-1]:

        tmp = pd.read_csv(i)
        tmp = tmp.replace(" ", np.nan)
        tmp = tmp.interpolate(method = 'values')
        tmp = tmp.fillna(0)

        for j in tqdm(range(len(tmp)-432)):
            train_data.append(np.array(tmp.loc[j:j + 431, ["swl", "inf", "sfw", "ecpc",
                                                           "tototf", "tide_level",
                                                           "fw_1018662", "fw_1018680",
                                                           "fw_1018683", "fw_1019630"]]).astype(float))

            train_label.append(np.array(tmp.loc[j + 432, ["wl_1018662", "wl_1018680",
                                                          "wl_1018683", "wl_1019630"]]).astype(float))

100%|██████████| 26064/26064 [00:18<00:00, 1389.04it/s]


In [7]:
train_data = np.array(train_data)
train_label = np.array(train_label)

print(train_data.shape)
print(train_label.shape)

(260640, 432, 10)
(260640, 4)


In [8]:
input_shape = (train_data[0].shape[0], train_data[0].shape[1])

model = Sequential()
model.add(GRU(256, input_shape=input_shape))
model.add(Dense(4, activation = 'relu'))

optimizer = tf.optimizers.RMSprop(0.001)

model.compile(optimizer=optimizer,loss='mse', metrics=['mae'])

In [9]:
with tf.device("/device:GPU:0"):
    model.fit(train_data, train_label, epochs=10, batch_size=256)

Epoch 1/5
1019/1019 [==============================] - 73s 66ms/step - loss: 42915.8711 - mae: 182.5115
Epoch 2/5
1019/1019 [==============================] - 67s 66ms/step - loss: 2975.6130 - mae: 24.9978
Epoch 3/5
1019/1019 [==============================] - 67s 66ms/step - loss: 926.7836 - mae: 11.5977
Epoch 4/5
1019/1019 [==============================] - 67s 66ms/step - loss: 561.6140 - mae: 9.1223
Epoch 5/5
1019/1019 [==============================] - 67s 66ms/step - loss: 434.9546 - mae: 8.2151


In [10]:
with tf.device("/device:GPU:0"):
    test_data = []
    test_label = []

    tmp = pd.read_csv(w_list[-1])
    tmp = tmp.replace(" ", np.nan)
    # 이전값을 사용
    tmp = tmp.fillna(method = 'pad')
    tmp = tmp.fillna(0)

    #tmp.loc[:, ["wl_1018662", "wl_1018680", "wl_1018683", "wl_1019630"]] = tmp.loc[:, ["wl_1018662", "wl_1018680", "wl_1018683", "wl_1019630"]]*100

    for j in tqdm(range(4032, len(tmp)-432)):
        test_data.append(np.array(tmp.loc[j:j + 431, ["swl", "inf", "sfw", "ecpc",
                                                        "tototf", "tide_level",
                                                        "fw_1018662", "fw_1018680",
                                                        "fw_1018683", "fw_1019630"]]).astype(float))

        test_label.append(np.array(tmp.loc[j + 432, ["wl_1018662", "wl_1018680",
                                                        "wl_1018683", "wl_1019630"]]).astype(float))

100%|██████████| 6912/6912 [00:05<00:00, 1367.36it/s]


In [11]:
test_data = np.array(test_data)
test_label = np.array(test_label)

print(test_data.shape)
print(test_label.shape)

(6912, 432, 10)
(6912, 4)


In [12]:
with tf.device("/device:GPU:0"):
    pred = model.predict(test_data)

216/216 [==============================] - 2s 9ms/step


In [13]:
pred = pd.DataFrame(pred)

In [14]:
sample_submission = pd.read_csv("Dataset/sample_submission.csv")

sample_submission["wl_1018662"] = pred[0]
sample_submission["wl_1018680"] = pred[1]
sample_submission["wl_1018683"] = pred[2]
sample_submission["wl_1019630"] = pred[3]

In [15]:
sample_submission.to_csv("Dataset/baseline.csv", index = False)

### score : 30.01091